In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import cv2
import math
import tqdm.notebook as tq
import numpy as np
import pandas as pd
import operator as op
from functools import reduce
from scipy.interpolate import interp1d
from google.colab.patches import cv2_imshow

In [ ]:
epsilon = 0.00001
n=2
m=2
gamma=4
img_name = '0a0bb7af0cab'
test_folder = '/content/drive/MyDrive/siim/'
dir="/content/drive/MyDrive/siim/"

In [ ]:
def add(epsilon , x1, x2):
    return (x1 + x2) / (1 + (x1 * x2) + epsilon)

def subtract(epsilon ,x1, x2):
    return (x1 - x2) / (1 - (x1 * x2) + epsilon)

def mult(epsilon, lamda, x):
    nom = ((1 + x) ** lamda) - ((1 - x) ** lamda)
    denom = ((1 + x) ** lamda) + ((1 - x) ** lamda)
    return nom / (denom+epsilon )

def fai(epsilon ,x):
    return 0.5 * np.log((1 + x) / (1 - x + epsilon) )

def norm(epsilon ,x):
    return np.abs(fai(epsilon,x))

def comb(n, r):
    r = min(r, n - r)
    numer = reduce(op.mul, range(n, n - r, -1), 1)
    denom = reduce(op.mul, range(1, r + 1), 1)
    return numer / denom

def colorAdd(q1,q2):
    res = []
    res.append(add(epsilon, q1[0] , q2[0]))
    res.append(add(epsilon, q1[1] , q2[1]))
    res.append(add(epsilon, q1[2] , q2[2]))
    return res


def colorSub(q1,q2):
    res = []
    res.append(subtract(epsilon, q1[0] , q2[0]))
    res.append(subtract(epsilon, q1[1] , q2[1]))
    res.append(subtract(epsilon, q1[2] , q2[2]))
    return res

def colorMult(lamda,q):
    res = []
    res.append(mult(epsilon, lamda, q[0]))
    res.append(mult(epsilon, lamda, q[1]))
    res.append(mult(epsilon, lamda, q[2]))
    return res

def colorNorm(q):
    res = 0
    res += fai(epsilon, q[0])**2
    res += fai(epsilon, q[1])**2
    res += fai(epsilon, q[2])**2
    return np.sqrt(res)

In [ ]:
class ImageEnh:
    def __init__(self, image, n, m, gamma):
        self.image = image
        self.n = n
        self.m = m
        self.gamma = gamma
        self.pixelMemberships = np.full((n, m, image.shape[0], image.shape[1]), -1000 , dtype=np.float)
        self.windowsMean = np.full((n, m), -1000 ,dtype=np.float)
        self.windowsCard = np.full((n, m), -1000 , dtype=np.float)
        self.pijMat = np.full((n, m, image.shape[0], image.shape[1]), -1000,dtype=np.float)
        self.windowsVariance = np.full((n, m), -1000,dtype=np.float)
        self.image = self.convertImgDown(self.image)

    def convertImgDown(self, image):
        mapping = interp1d([0, 255], [-1, 1])
        image = mapping(image)
        #         for x in range(self.image.shape[0]):
        #             for y in range(self.image.shape[1]):
        #                 self.image[x][y] = (((self.image[x][y]) * (2)) / (255)) - 1
        return image

    def convertImgUp(self, img):
        num1 = np.abs(np.min(img))
        num2 = np.abs(np.max(img))
        mapping = interp1d([-1*max(num1,num2), max(num1,num2)], [0, 255])
        img = mapping(img)
        #         new_img = np.zeros((self.image.shape[0] , self.image.shape[1]))
        #         for x in range(self.image.shape[0]):
        #             for y in range(self.image.shape[1]):
        #                 new_img[x][y] = (((img[x][y] + 1) * (255)) / (2))
        return img

    def qxi(self, i, x):
        x0 = 0
        x1 = self.image.shape[0]
        nCi = comb(self.n, i)
        nom = ((x - x0) ** i) * ((x1 - x) ** (self.n - i))
        denom = (x1 - x0) ** self.n
        ans = nCi * nom / denom
        # if ans > 1 or ans < 0:
        #     print('Error in qxi : ', ans)
        return ans

    def qyj(self, j, y):
        y0 = 0
        y1 = self.image.shape[1]
        nCi = comb(self.m, j)
        nom = (np.power((y - y0),j)) * (np.power((y1 - y) , (self.m - j)))
        denom = (y1 - y0) ** self.m
        ans = nCi * nom / denom
        # if ans > 1 or ans < 0:
        #     print('Error in qyj : ', ans)
        return ans

    def pij(self, i, j, x, y):
        if self.pijMat[i][j][x][y] == -1000:
            ans = self.qxi(i, x) * self.qyj(j, y)
            # if ans > 1 or ans < 0:
            #     print('Error in pij : ', i, j, ans)
            self.pijMat[i][j][x][y] = ans
        return self.pijMat[i][j][x][y]

    def membership(self, i, j, x, y):
        if self.pixelMemberships[i][j][x][y] == -1000:
            nom = self.pij(i, j, x, y) ** self.gamma
            denom = 0
            for idx1 in range(self.n):
                for idx2 in range(self.m):
                    denom += np.power(self.pij(idx1, idx2, x, y), self.gamma)
            ans = nom / (denom + epsilon)
            if ans > 1 or ans < 0 or math.isnan(ans):
                print('Error in membership : ', denom)
            self.pixelMemberships[i][j][x][y] = ans
        return self.pixelMemberships[i][j][x][y]

    def windowCard(self, i, j):
        if self.windowsCard[i][j] == -1000:
            card = 0.0
            for x in range(self.image.shape[0]):
                for y in range(self.image.shape[1]):
                    card += self.membership(i, j, x, y)
            self.windowsCard[i][j] = card
        return self.windowsCard[i][j]

    def windowMean(self, i, j):
        if self.windowsMean[i][j] == -1000:
            card = self.windowCard(i, j)
            mean = 0.0
            for x in range(self.image.shape[0]):
                for y in range(self.image.shape[1]):
                    mean = add(epsilon, mean, mult(epsilon, self.membership(i, j, x, y) / card, self.image[x][y]))
            self.windowsMean[i][j] = mean
        return self.windowsMean[i][j]

    def windowVar(self, i, j):
        if self.windowsVariance[i][j] == -1000:
            var = 0.0
            card = self.windowCard(i, j)
            mean = self.windowMean(i, j)
            for x in range(self.image.shape[0]):
                for y in range(self.image.shape[1]):
                    memship = self.membership(i, j, x, y)
                    nom = memship * (norm(epsilon, subtract(epsilon, self.image[x][y], mean)) ** 2)
                    denom = card
                    var += nom / denom
            self.windowsVariance[i][j] = var
        return self.windowsVariance[i][j]

    def enhanceImage(self):
        image_copy = np.zeros((self.image.shape[0], self.image.shape[1]))
        sigma = np.sqrt(1/3)
        for i in range(self.n):
            for j in range(self.m):
                mean = self.windowMean(i, j)
                variance = np.sqrt(self.windowVar(i, j))
                for x in range(self.image.shape[0]):
                    for y in range(self.image.shape[1]):
                        left = sigma / variance
                        psi = mult(epsilon, left, subtract(epsilon,self.image[x][y], mean))
                        memship = self.membership(i, j, x, y)
                        image_copy[x][y] += mult(epsilon, memship, psi)
        image_copy = self.convertImgUp(image_copy)
        return image_copy

In [ ]:
class ColoredImageEnh:
    def __init__(self, image, n, m, gamma):
        """
        Initializing the model and it's variables
        :param image: image to be enhanced
        :param n: number of windows in width
        :param m: number of windows in height
        :param gamma: fuzzification coffecient
        """
        self.image = image
        self.n = n
        self.m = m
        self.gamma = gamma

        # Membership matrix of pixels to each window
        self.pixelMemberships = np.full((n, m, image.shape[0], image.shape[1]), -1000, dtype=np.float)
        # Mean of each window
        self.windowsMean = np.full((n, m), -1000, dtype=np.float)
        # Cardinality of each window
        self.windowsCard = np.full((n, m), -1000, dtype=np.float)
        self.pijMat = np.full((n, m, image.shape[0], image.shape[1]), -1000, dtype=np.float)
        # Variance of each window
        self.windowsVariance = np.full((n, m), -1000, dtype=np.float)

        self.image = self.convertImgDown(self.image)

        # Luminosity matrix of colored image
        self.lum = np.full((image.shape[0] , image.shape[1]) , -1000 , dtype=np.float)

    def convertImgDown(self, image):
        """
        Mapping pixel values from interval [0 , 255] to [-1 , 1]
        :param image: image needed to be converted
        :return: image after mapping it to interval [-1 , 1]
        """
        mapping = interp1d([0, 255], [-1, 1])
        image = mapping(image)
        return image

    def convertImgUp(self, img):
        """
        Mapping pixel values from interval [-1 , 1] to interval [0 , 255]
        :param img: image needed to be converted
        :return: image after mapping
        """
        mapping = interp1d([np.min(img), np.max(img)], [0, 255])
        img = mapping(img)
        return img

    def imageLuminosity(self,i,j):
        """
        Calculating the Luminosity of colored image at index i , j
        :param i: row index
        :param j: column index
        :return: The luminosity value at index [i j]
        """
        if self.lum[i][j] == -1000:
            for x in range(self.image.shape[0]):
                for y in range(self.image.shape[1]):
                    temp = add(epsilon, self.image[x][y][2] , self.image[x][y][1])
                    temp = add(epsilon, temp , self.image[x][y][0])
                    self.lum[x][y] = mult(epsilon, 1/3 , temp)
        return self.lum[i][j]


    def qxi(self, i, x):
        """
        Calculating formula:
        :param i:
        :param x:
        :return:
        """
        x0 = 0
        x1 = self.image.shape[0]
        nCi = comb(self.n, i)
        nom = ((x - x0) ** i) * ((x1 - x) ** (self.n - i))
        denom = (x1 - x0) ** self.n
        ans = nCi * nom / denom
        return ans

    def qyj(self, j, y):
        y0 = 0
        y1 = self.image.shape[1]
        nCi = comb(self.m, j)
        nom = (np.power((y - y0), j)) * (np.power((y1 - y), (self.m - j)))
        denom = (y1 - y0) ** self.m
        ans = nCi * nom / denom
        return ans

    def pij(self, i, j, x, y):
        if self.pijMat[i][j][x][y] == -1000:
            ans = self.qxi(i, x) * self.qyj(j, y)
            self.pijMat[i][j][x][y] = ans
        return self.pijMat[i][j][x][y]

    def membership(self, i, j, x, y):
        """
        Calculating the membership of pixel [x][y] to a window i,j
        :param i: window row index
        :param j: window column index
        :param x: pixel row index
        :param y: pixel column index
        :return: Membership value
        """
        if self.pixelMemberships[i][j][x][y] == -1000:
            nom = self.pij(i, j, x, y) ** self.gamma
            denom = 0
            for idx1 in range(self.n):
                for idx2 in range(self.m):
                    denom += np.power(self.pij(idx1, idx2, x, y), self.gamma)
            ans = nom / (denom + epsilon)
            self.pixelMemberships[i][j][x][y] = ans
        return self.pixelMemberships[i][j][x][y]

    def windowCard(self, i, j):
        """
        Calculating the cardinality of window i,j
        :param i:
        :param j:
        :return: window card value
        """
        if self.windowsCard[i][j] == -1000:
            card = 0.0
            for x in range(self.image.shape[0]):
                for y in range(self.image.shape[1]):
                    card += self.membership(i, j, x, y)
            self.windowsCard[i][j] = card
        return self.windowsCard[i][j]

    def windowMean(self, i, j):
        """
        Calculating the mean of window i,j
        :param i:
        :param j:
        :return: Mean value
        """
        if self.windowsMean[i][j] == -1000:
            card = self.windowCard(i, j)
            mean = 0.0
            for x in range(self.image.shape[0]):
                for y in range(self.image.shape[1]):
                    mean = add(epsilon, mean, mult(epsilon, self.membership(i, j, x, y) / card, self.imageLuminosity(x,y)))
            self.windowsMean[i][j] = mean
        return self.windowsMean[i][j]

    def windowVar(self, i, j):
        """
        Calculating the squared Variance of a window
        :param i:
        :param j:
        :return: Squared variance value
        """
        if self.windowsVariance[i][j] == -1000:
            var = 0.0
            card = self.windowCard(i, j)
            mean = self.windowMean(i, j)
            for x in range(self.image.shape[0]):
                for y in range(self.image.shape[1]):
                    memship = self.membership(i, j, x, y)
                    nom = memship * (norm(epsilon, subtract(epsilon, self.imageLuminosity(x,y), mean)) ** 2)
                    denom = card
                    var += nom / denom
            self.windowsVariance[i][j] = var
        return self.windowsVariance[i][j]

    def imageEnhance(self):
        """
        Iterating over the channels , pixel and windows and calculate the new image after enhancing
        :return: Enhanced image after converting it to interval[0,255]
        """
        final_img = np.zeros((self.image.shape[0] , self.image.shape[1] , 3) , dtype=np.float)
        sigma = np.sqrt(1/3)
        for chn in range(3):
            for i in range(self.n):
                for j in range(self.m):
                    var = np.sqrt(self.windowVar(i,j))
                    mean = self.windowMean(i,j)
                    for x in range(self.image.shape[0]):
                        for y in range(self.image.shape[1]):
                            memship = self.membership(i, j, x, y)
                            final_img[x][y][chn] += mult(epsilon, (memship*sigma/var) , subtract(epsilon, self.image[x][y][chn] , mean))

        final_img = self.convertImgUp(final_img)

        return final_img

In [ ]:
def colored_enhancing(filepath, gray_tarpath):
    img = cv2.imread(filepath)
    #print(img.shape)
    imgEnh = ColoredImageEnh(img, n, m, gamma)
    final_image = imgEnh.imageEnhance()
    final_image = np.array(final_image, dtype=np.uint8)
    #cv2_imshow(img)
    #cv2_imshow(final_image)
    #cv2.imshow('before converting', img)
    #cv2.imshow('final', final_image)
    cv2.imwrite(gray_tarpath, final_image)
    #print(final_image)
    #cv2.waitKey(1)
    #cv2.destroyAllWindows()

def gray_enhancing(filepath, colored_tarpath):
    img = cv2.imread(filepath)
    #print(img.shape)
    img = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
    imgEnh = ImageEnh(img ,n,m,gamma)
    final_image = imgEnh.enhanceImage()
    final_image = np.array(final_image , dtype = np.uint8)
    #cv2_imshow(img)
    #cv2_imshow(final_image)
    #cv2_imshow('before converting' , img)
    #cv2_imshow('final' , final_image)
    cv2.imwrite(colored_tarpath, final_image)
    #cv2.waitKey(1)
    #cv2.destroyAllWindows()

In [ ]:
info=pd.read_csv(dir+"summary/train_summary.csv")
info.head()

,Unnamed: 0,image_id,study_id,pneumonia,pneumonia_class,height,width,boxes_x,image_label,image_label_id,filepath,data,id,boxes_y,label,StudyInstanceUID,image_level
0,0,b9175a64ad09,587699c00f43,Indeterminate Appearance,2,2618,3005,"[{'x': 2244.66714, 'y': 1575.60367, 'width': 2...",opacity,1,/content/drive/MyDrive/siim/tmp/train/b9175a64...,train,b9175a64ad09_image,"[{'x': 2244.66714, 'y': 1575.60367, 'width': 2...",opacity 1 2244.66714 1575.60367 2504.06533 178...,587699c00f43,opacity
1,1,8b2544f4361c,7d04273b5886,Typical Appearance,1,3480,4248,"[{'x': 597.81637, 'y': 1014.4836, 'width': 515...",opacity,1,/content/drive/MyDrive/siim/tmp/train/8b2544f4...,train,8b2544f4361c_image,"[{'x': 597.81637, 'y': 1014.4836, 'width': 515...",opacity 1 597.81637 1014.4836 1113.13638 2448....,7d04273b5886,opacity
2,2,1808bdef30dd,7fd0efde20b7,Indeterminate Appearance,2,2336,2836,"[{'x': 566.36272, 'y': 480.19347, 'width': 483...",opacity,1,/content/drive/MyDrive/siim/tmp/train/1808bdef...,train,1808bdef30dd_image,"[{'x': 566.36272, 'y': 480.19347, 'width': 483...",opacity 1 566.36272 480.19347 1049.38085 1415....,7fd0efde20b7,opacity
3,3,66c5884489fb,df21842cfe5e,Typical Appearance,1,2320,2832,"[{'x': 443.42472, 'y': 879.07294, 'width': 467...",opacity,1,/content/drive/MyDrive/siim/tmp/train/66c58844...,train,66c5884489fb_image,"[{'x': 443.42472, 'y': 879.07294, 'width': 467...",opacity 1 443.42472 879.07294 911.18833 1806.5...,df21842cfe5e,opacity
4,4,95b239a10358,1d667e9556ad,Typical Appearance,1,2520,3032,"[{'x': 82.69968, 'y': 889.10112, 'width': 836....",opacity,1,/content/drive/MyDrive/siim/tmp/train/95b239a1...,train,95b239a10358_image,"[{'x': 82.69968, 'y': 889.10112, 'width': 836....",opacity 1 82.69968 889.10112 918.79295 2211.87...,1d667e9556ad,opacity


In [ ]:
info["fgfp"]=info.apply(lambda x:x.filepath.replace('tmp','fgtmp'),axis=1)
info["fcfp"]=info.apply(lambda x:x.filepath.replace('tmp','fctmp'),axis=1)
info.to_csv(dir+"summary/info_summary.csv")
info.head()

,Unnamed: 0,image_id,study_id,pneumonia,pneumonia_class,height,width,boxes_x,image_label,image_label_id,filepath,data,id,boxes_y,label,StudyInstanceUID,image_level,fgfp,fcfp
0,0,b9175a64ad09,587699c00f43,Indeterminate Appearance,2,2618,3005,"[{'x': 2244.66714, 'y': 1575.60367, 'width': 2...",opacity,1,/content/drive/MyDrive/siim/tmp/train/b9175a64...,train,b9175a64ad09_image,"[{'x': 2244.66714, 'y': 1575.60367, 'width': 2...",opacity 1 2244.66714 1575.60367 2504.06533 178...,587699c00f43,opacity,/content/drive/MyDrive/siim/fgtmp/train/b9175a...,/content/drive/MyDrive/siim/fctmp/train/b9175a...
1,1,8b2544f4361c,7d04273b5886,Typical Appearance,1,3480,4248,"[{'x': 597.81637, 'y': 1014.4836, 'width': 515...",opacity,1,/content/drive/MyDrive/siim/tmp/train/8b2544f4...,train,8b2544f4361c_image,"[{'x': 597.81637, 'y': 1014.4836, 'width': 515...",opacity 1 597.81637 1014.4836 1113.13638 2448....,7d04273b5886,opacity,/content/drive/MyDrive/siim/fgtmp/train/8b2544...,/content/drive/MyDrive/siim/fctmp/train/8b2544...
2,2,1808bdef30dd,7fd0efde20b7,Indeterminate Appearance,2,2336,2836,"[{'x': 566.36272, 'y': 480.19347, 'width': 483...",opacity,1,/content/drive/MyDrive/siim/tmp/train/1808bdef...,train,1808bdef30dd_image,"[{'x': 566.36272, 'y': 480.19347, 'width': 483...",opacity 1 566.36272 480.19347 1049.38085 1415....,7fd0efde20b7,opacity,/content/drive/MyDrive/siim/fgtmp/train/1808bd...,/content/drive/MyDrive/siim/fctmp/train/1808bd...
3,3,66c5884489fb,df21842cfe5e,Typical Appearance,1,2320,2832,"[{'x': 443.42472, 'y': 879.07294, 'width': 467...",opacity,1,/content/drive/MyDrive/siim/tmp/train/66c58844...,train,66c5884489fb_image,"[{'x': 443.42472, 'y': 879.07294, 'width': 467...",opacity 1 443.42472 879.07294 911.18833 1806.5...,df21842cfe5e,opacity,/content/drive/MyDrive/siim/fgtmp/train/66c588...,/content/drive/MyDrive/siim/fctmp/train/66c588...
4,4,95b239a10358,1d667e9556ad,Typical Appearance,1,2520,3032,"[{'x': 82.69968, 'y': 889.10112, 'width': 836....",opacity,1,/content/drive/MyDrive/siim/tmp/train/95b239a1...,train,95b239a10358_image,"[{'x': 82.69968, 'y': 889.10112, 'width': 836....",opacity 1 82.69968 889.10112 918.79295 2211.87...,1d667e9556ad,opacity,/content/drive/MyDrive/siim/fgtmp/train/95b239...,/content/drive/MyDrive/siim/fctmp/train/95b239...


In [ ]:
import os
os.makedirs('/content/drive/MyDrive/siim/fgtmp/train', exist_ok=True)
os.makedirs('/content/drive/MyDrive/siim/fgtmp/test', exist_ok=True)

os.makedirs('/content/drive/MyDrive/siim/fctmp/train', exist_ok=True)
os.makedirs('/content/drive/MyDrive/siim/fctmp/test', exist_ok=True)

In [ ]:
info.shape

(6334, 19)

In [ ]:
# first time 893 seccess
# second time 1160 seccess
# third time 650 success
# forth time 761 success
# fifth time 382 success
# sixth time 91 success
# seventh time 996 success
# eighth time 987 success
# ninth time 416 success
for i in tq.tqdm(range(5918,info.shape[0])):
    row=info.iloc[i]
    filepath=row.filepath
    gray_tarpath=row.fgfp
    #print(gray_tarpath)
    colored_tarpath=row.fcfp
    #print(colored_tarpath)d
    gray_enhancing(filepath, gray_tarpath)
    colored_enhancing(filepath, colored_tarpath)

  0%|          | 0/416 [00:00<?, ?it/s]